In [2]:
!pip install polars beautifulsoup4 splinter selenium

In [3]:
!pip install pint

In [4]:
import polars as pl
import pathlib
import re
from bs4 import BeautifulSoup
# Will use firefox browser
from splinter import Browser
import time
import pint

## Prepping the data. Do not include in main

In [5]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [6]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources')

In [7]:
data_path = f"{resources_path}/MIF_search.csv"
df_data = pl.read_csv(data_path)
df_data.glimpse()

Rows: 8
Columns: 1
$ steel <str> 'AISI-SAE 4037', 'AISI-SAE 1042', 'AISI-SAE 1080', 'AISI-SAE 1335', 'AISI 4047', 'AISI 52100', 'AISI 1049', 'AISI-SAE 9254'



## Visit the Website

In [8]:
browser = Browser('firefox')
# https://www.makeitfrom.com/material-group/Wrought-Alloy-Steel-SAE-AISI
# OR
# https://www.makeitfrom.com/material-group/Wrought-Carbon-Or-Non-Alloy-Steel
base_url = "https://www.makeitfrom.com/material-group/"
url_alloy_steel = "Wrought-Alloy-Steel-SAE-AISI"
url_carbon_steel = "Wrought-Carbon-Or-Non-Alloy-Steel"

### Rename some steel present in MIF that were not searching correctly

In [8]:
df_data

searchable
str
"""AISI-SAE 1335"""
"""AISI-SAE 4027"""
"""AISI 1055"""
"""AISI-SAE 1042"""
"""AISI 6145"""
…
"""AISI 4047"""
"""AISI-SAE 1080"""
"""AISI-SAE 1040"""
